In [13]:
import duckdb
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import shutil # Import the shutil module


In [14]:
# First cell: Read parquet and get descriptive statistics
conn = duckdb.connect()
conn.sql("CREATE TABLE results AS SELECT * FROM read_parquet('results.parquet')")
conn.sql("SELECT * FROM results").show()


┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────────┬────────────────────────────────────────────┬────────────┬────────────────┬─────────────┬────────────────────┬─────────────┬───────────────┐
│                                                     path                                                     │      dataset       │                  col_name                  │    algo    │ amount_of_rows │ run_time_ms │ compression_factor │ num_strings │ original_size │
│                                                   varchar                                                    │      varchar       │                  varchar                   │  varchar   │     int64      │   double    │       double       │    int64    │     int64     │
├──────────────────────────────────────────────────────────────────────────────────────────────────────────────┼────────────────────┼────────────────────────────────────────────┼

In [15]:
# Create the results table with dataset_column
conn.execute("""
CREATE OR REPLACE TABLE results AS
SELECT *,
       dataset || '_' || col_name AS dataset_column
FROM results
""")

In [16]:
# Sort the results table by dataset_column
conn.execute("""
CREATE OR REPLACE TABLE results AS
SELECT *
FROM results
ORDER BY dataset_column
""")

In [17]:
import plotly.express as px
import os
import shutil
import pandas as pd # Assuming df is a pandas DataFrame, ensure pandas is imported

# Assume df is loaded somewhere above this script
def save_plots(df, file_name):
    plots_dir = "plots_duckdb"
    # # Remove the directory and its contents if it exists
    # if os.path.exists(plots_dir):
    #     shutil.rmtree(plots_dir)
    #     print(f"Removed existing directory: '{plots_dir}'")

    os.makedirs(plots_dir, exist_ok=True)

    figures = [] # List to store figures

    for i in range(0, len(df), 200):
        start_row = i
        end_row = min(i + 200, len(df))
        # Ensure df_chunk is created correctly, especially if df index is not standard 0..N-1
        # If df has a standard integer index, this is fine.
        # If not, consider using iloc for integer position slicing:
        # df_chunk = df.iloc[start_row:end_row]
        df_chunk = df.loc[start_row:end_row-1] # Using loc as in original code

        # Create the plot for the current chunk
        fig = px.bar(
            df_chunk,
            x="dataset_column",
            y="compression_factor",
            color="algo",
            barmode="group",
            hover_data=["path"],
            color_discrete_map={
                "basic_fsst": "#636EFA",
                "fsst_plus": "#EF553B",
                "dictionary": "#00CC96"
            },
            category_orders={"algo": ["basic_fsst", "fsst_plus", "dictionary"]},
            title=f"Compression Factor Comparison (Rows {start_row}-{end_row-1})"
        )
        figures.append(fig) # Add the figure to the list

    # Define the single output filename
    output_filename = os.path.join(plots_dir, f"{file_name}.html")
    #Remove the file if it exists
    if os.path.exists(output_filename):
        os.remove(output_filename)
    # Write all figures to the single HTML file
    with open(output_filename, 'w') as f:
        # Write HTML header with Plotly JS CDN link
        f.write("<html><head><meta charset='utf-8' /><script src='https://cdn.plot.ly/plotly-latest.min.js'></script></head><body>\n")

        # Write each figure's HTML div
        for fig in figures:
            f.write(fig.to_html(full_html=False, include_plotlyjs=False)) # Write fig div
            f.write("<hr>\n") # Add a horizontal rule between plots for separation

        # Write HTML closing tags
        f.write("</body></html>\n")


    print(f"All plots saved in a single file: '{output_filename}'")

In [18]:

# First, create a table for dataset_columns with all 3 algorithms
conn.execute("""
CREATE OR REPLACE TABLE all_three_algos AS
SELECT dataset_column
FROM results
GROUP BY dataset_column
HAVING COUNT(*) = 3
""")

# Create pivoted table for compression factors
conn.execute("""
CREATE OR REPLACE TABLE pivoted AS
SELECT 
    dataset_column,
    MAX(CASE WHEN algo = 'basic_fsst' THEN compression_factor END) AS basic_fsst,
    MAX(CASE WHEN algo = 'fsst_plus' THEN compression_factor END) AS fsst_plus
FROM results
WHERE algo IN ('basic_fsst', 'fsst_plus')
GROUP BY dataset_column
""")

# Create fsst_plus_better table
conn.execute("""
CREATE OR REPLACE TABLE fsst_plus_better AS
SELECT r.*
FROM results r
JOIN all_three_algos a ON r.dataset_column = a.dataset_column
JOIN pivoted p ON r.dataset_column = p.dataset_column
WHERE p.fsst_plus > p.basic_fsst
""")

# Create fsst_plus_worse table
conn.execute("""
CREATE OR REPLACE TABLE fsst_plus_worse AS
SELECT r.*
FROM results r
JOIN all_three_algos a ON r.dataset_column = a.dataset_column
JOIN pivoted p ON r.dataset_column = p.dataset_column
WHERE p.fsst_plus < p.basic_fsst
""")

# Convert to pandas and save plots
all_results = conn.execute("SELECT * FROM results").df()
fsst_plus_better = conn.execute("SELECT * FROM fsst_plus_better").df()
fsst_plus_worse = conn.execute("SELECT * FROM fsst_plus_worse").df()

save_plots(all_results, "all_results")
save_plots(fsst_plus_better, "fsst_plus_better")
save_plots(fsst_plus_worse, "fsst_plus_worse")

All plots saved in a single file: 'plots_duckdb/all_results.html'
All plots saved in a single file: 'plots_duckdb/fsst_plus_better.html'
All plots saved in a single file: 'plots_duckdb/fsst_plus_worse.html'
